In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [11]:

data_dir = "D:\STUDY\Sumit\lectures sumit\college\sem 6\dyslexia\Dataset Dyslexia_Password WanAsy321\Gambo\Train"  

categories = ["normal", "reversal", "corrected"]

img_size = (64, 64)  


In [12]:
X = []  
y = []  
label_map = {"normal": 0, "reversal": 1, "corrected": 2}


for category in categories:
    folder_path = os.path.join(data_dir, category)
    
    if not os.path.exists(folder_path):
        print(f"Folder missing: {folder_path}")
        continue
    
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

        
        img = load_img(img_path, target_size=img_size, color_mode='grayscale')
        img_array = img_to_array(img) / 255.0  
        
        X.append(img_array)
        y.append(label_map[category])


X = np.array(X)
y = np.array(y)

print(f"Total images loaded: {X.shape[0]}")


Total images loaded: 151649


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [15]:
from sklearn.model_selection import train_test_split
# Split dataset (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)

print(f"Training samples: {X_train.shape[0]}, Validation samples: {X_val.shape[0]}")

Training samples: 121319, Validation samples: 30330


In [16]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
3033/3033 [==============================] - 310s 102ms/step - loss: 0.1850 - accuracy: 0.9319 - val_loss: 0.0971 - val_accuracy: 0.9656
Epoch 2/10
3033/3033 [==============================] - 330s 109ms/step - loss: 0.0889 - accuracy: 0.9674 - val_loss: 0.0596 - val_accuracy: 0.9780
Epoch 3/10
3033/3033 [==============================] - 327s 108ms/step - loss: 0.0597 - accuracy: 0.9793 - val_loss: 0.0453 - val_accuracy: 0.9853
Epoch 4/10
3033/3033 [==============================] - 338s 112ms/step - loss: 0.0440 - accuracy: 0.9847 - val_loss: 0.0474 - val_accuracy: 0.9838
Epoch 5/10
3033/3033 [==============================] - 331s 109ms/step - loss: 0.0352 - accuracy: 0.9876 - val_loss: 0.0395 - val_accuracy: 0.9866
Epoch 6/10
3033/3033 [==============================] - 312s 103ms/step - loss: 0.0292 - accuracy: 0.9894 - val_loss: 0.0360 - val_accuracy: 0.9898
Epoch 7/10
3033/3033 [==============================] - 340s 112ms/step - loss: 0.0247 - accuracy: 0.9913 - val_

In [20]:
# Load test dataset
test_data_dir = "D:\STUDY\Sumit\lectures sumit\college\sem 6\dyslexia\Dataset Dyslexia_Password WanAsy321\Gambo\Test"

X_test, y_test = [], []
for category in categories:
    folder_path = os.path.join(test_data_dir, category)
    
    if not os.path.exists(folder_path):
        print(f"Folder missing: {folder_path}")
        continue
    
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

        # Load image in grayscale, resize, and normalize
        img = load_img(img_path, target_size=img_size, color_mode='grayscale')
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        
        X_test.append(img_array)
        y_test.append(label_map[category])

# Convert lists to NumPy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Convert labels to categorical format
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

print(f"Total test images loaded: {X_test.shape[0]}")


Total test images loaded: 56723


In [21]:
# Evaluate model on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


1773/1773 [==============================] - 34s 19ms/step - loss: 1.4706 - accuracy: 0.8637
Test Accuracy: 86.37%


In [22]:
model.save("char_recognition_model.h5")